In [1]:
import pandas as pd
from io import StringIO
import dateutil.parser as dp
import airtable

In [2]:
fed = pd.read_csv("../sample_data/Federal.csv")
ab = pd.read_csv("../sample_data/Alberta.csv")
nu = pd.read_csv("../sample_data/Nunuvat.csv")
bc = pd.read_csv("../sample_data/British_Columbia.csv")
yk = pd.read_csv("../sample_data/Yukon.csv")

In [33]:
col_df = airtable.get_df()

In [34]:
col_df.set_index("organization", inplace=True)

In [35]:
del col_df["id"]

In [36]:
rename_dict = {}
for new_name in col_df.columns:
    old_name = col_df[new_name]["federal"]
    if old_name == old_name:
        rename_dict[old_name] = new_name

In [38]:
# from https://stackoverflow.com/a/4766400

tz_str = '''-12 Y
-11 X NUT SST
-10 W CKT HAST HST TAHT TKT
-9 V AKST GAMT GIT HADT HNY
-8 U AKDT CIST HAY HNP PST PT
-7 T HAP HNR MST PDT
-6 S CST EAST GALT HAR HNC MDT
-5 R CDT COT EASST ECT EST ET HAC HNE PET
-4 Q AST BOT CLT COST EDT FKT GYT HAE HNA PYT
-3 P ADT ART BRT CLST FKST GFT HAA PMST PYST SRT UYT WGT
-2 O BRST FNT PMDT UYST WGST
-1 N AZOT CVT EGT
0 Z EGST GMT UTC WET WT
1 A CET DFT WAT WEDT WEST
2 B CAT CEDT CEST EET SAST WAST
3 C EAT EEDT EEST IDT MSK
4 D AMT AZT GET GST KUYT MSD MUT RET SAMT SCT
5 E AMST AQTT AZST HMT MAWT MVT PKT TFT TJT TMT UZT YEKT
6 F ALMT BIOT BTT IOT KGT NOVT OMST YEKST
7 G CXT DAVT HOVT ICT KRAT NOVST OMSST THA WIB
8 H ACT AWST BDT BNT CAST HKT IRKT KRAST MYT PHT SGT ULAT WITA WST
9 I AWDT IRKST JST KST PWT TLT WDT WIT YAKT
10 K AEST ChST PGT VLAT YAKST YAPT
11 L AEDT LHDT MAGT NCT PONT SBT VLAST VUT
12 M ANAST ANAT FJT GILT MAGST MHT NZST PETST PETT TVT WFT
13 FJST NZDT
11.5 NFT
10.5 ACDT LHST
9.5 ACST
6.5 CCT MMT
5.75 NPT
5.5 SLT
4.5 AFT IRDT
3.5 IRST
-2.5 HAT NDT
-3.5 HNT NST NT
-4.5 HLV VET
-9.5 MART MIT'''

tzd = {}
for tz_descr in map(str.split, tz_str.split('\n')):
    tz_offset = int(float(tz_descr[0]) * 3600)
    for tz_code in tz_descr[1:]:
        tzd[tz_code] = tz_offset

In [39]:
def parse_fed_time(x):
    y = x.split(" ")
    clean_str = y[0] + " " + y[1] + " " + y[-1][1:-1]
    return dp.parse(clean_str, tzinfos=tzd)

In [40]:
sample_data_fed = fed.query("language == 'English'")[list(rename_dict.keys())].rename(columns=rename_dict)

In [41]:
sample_data_fed["closing_date"] = sample_data_fed["closing_date"].map(parse_fed_time)

In [42]:
sample_data = pd.DataFrame(columns=col_df.keys())
sample_data = pd.concat([sample_data, sample_data_fed], sort=True)
sample_data.index = range(len(sample_data))

In [43]:
sample_data.head()

,GSIN_code,URL,buyer_contact,buyer_name,closing_date,location_of_work,province_territory_of_work,published_date,tender_description,title
0,"JX1990A - Ships and Vessels (large) - Repair, ...",NaN,"DeBlois, Vincent, vincent.deblois@tpsgc-pwgsc....",Fisheries and Oceans Canada,2019-09-12 14:00:00-04:00,NaN,Quebec,2019-08-16,Trade Agreement: Canadian Free Trade Agreement...,CCGS Amundsen -Dry Dock and refit-Fall 2019 (F...
1,"E111A - Oil Spill Collection, JX1990H - Ships ...",NaN,"Richards, Shazia, shazia.khan@tpsgc-pwgsc.gc.c...",Fisheries and Oceans Canada,2019-09-12 14:00:00-04:00,NaN,"British Columbia, Newfoundland and Labrador, N...",2019-06-21,Trade Agreement: NAFTA / CFTA / FTAs with Peru...,EREP: Self-Propelled Advancing Skimmer (F7047-...
2,T004KA - Social Media Monitoring,NaN,"Westall, Susan, susan.westall@tpsgc-pwgsc.gc.c...",Public Works and Government Services Canada,2020-01-31 14:00:00-05:00,NaN,"Alberta, British Columbia, Manitoba, National ...",2014-01-20,Trade Agreement: Canadian Free Trade Agreement...,Social Media Monitoring (EN578-141760/B)
3,N7030 - ADP Software,NaN,"Niyonambaza (SMS div), Audace, audace.niyonamb...",Public Works and Government Services Canada,2023-10-02 14:00:00-04:00,NaN,National Capital Region,2013-07-03,Trade Agreement: NONE Tendering Procedures: Al...,Software Licensing Supply Arrangement (EN578-1...
4,"T002AA - Fabrication of Exhibits, T002AB - Cus...",NaN,"Kalp, Lynn, 613-938-5803, 866-246-6893, 111 Wa...",Parks Canada,2019-10-01 14:00:00-04:00,NaN,Canada,2014-02-19,"Request for Supply Arrangement, Parks Canada A...",RFSA -Exhibit Design - Parks Canada - National...


In [45]:
sample_data.to_json("../sample_data/sample_data.json", orient="records")